

# Study on Generating Novel Antibiotics Using Graph Neural Networks (GNNs)

*Predicting and Designing New Antibiotics using Graph Neural Networks (GNNs)*

*Christopher L. Gaughan, Ph.D.*

## Introduction

The Centers for Disease Control and Prevention (CDC) has stated that the post-antibiotic era is already here. This is because bacteria have developed resistance to every antibiotic that has been developed, and common drugs like penicillin are no longer effective. The discovery of new antibiotic molecules is critical to combat the rising threat of resistant bacterial strains. This study aims to leverage **Graph Neural Networks (GNNs)** to discover and design new molecular structures with potential antibacterial properties inspired by recent breakthroughs in AI-driven drug discovery, such as the discovery of **Halicin**, an antibiotic identified using neural networks.

> "CDC's 2019 AR Threats Report includes national death and infection estimates that underscore the continued threat of AR in the United States. More than 2.8 million antimicrobial-resistant infections occur in the U.S. each year, and more than 35,000 people die as a result. When C. diff is added to these, the U.S. toll of all the threats in the report exceeds 3 million infections and 48,000 deaths."

*2019 Antibiotic Resistance Threats Report*


Mechanisms of Various Antibiotics

![Antibiotic-Mechanisms](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5672523/bin/JOACP-33-300-g002.jpg)

*J Anaesthesiol Clin Pharmacol. 2017 Jul-Sep; 33(3): 300–305.*


## Goals

Our objectives in this study are:
1. **Develop a Predictive GNN Model**: Build and train a GNN to predict the antibacterial effectiveness of molecular structures based on their graph representations. Initially, the study will be brief in scope, focusing only on a small subset of known, active antibiotics for the training of our model.
   
2. **Generate Novel Antibiotic Molecules**: Use generative models like **Variational Autoencoders (VAEs)** or **Reinforcement Learning-based GNNs** to propose new molecular structures that could act as antibiotics.

3. **AWS SageMaker Integration**: We will scale up the training set of known antibacterial compounds and deploy our model on AWS SageMaker to create a scalable endpoint. This will allow researchers and practitioners to input new molecular structures and receive predictions about their antibacterial activity, enabling large-scale testing.

4. Optimization and Iteration: Continue refining the generated molecules using graph optimization techniques to ensure they possess drug-like properties while enhancing their antibacterial potential.

## Methods

### Dataset Preparation
We begin by transforming molecular data into graph representations, with **adjacency matrices** representing atomic bonds and **node features** representing atomic properties (e.g., atomic number, charge, molecular weight). The dataset consists of known antibiotic compounds, which will be used to train our GNN model.

### GNN Architecture
The Graph Neural Network (GNN) encodes molecular graphs into vector representations by passing information (or "messages") between atoms and bonds. These graph embeddings are then used to predict whether a molecule is antibacterial or non-antibacterial, providing the foundation for further molecular generation.

### AWS SageMaker Scaling
After developing a robust model, the next step is to deploy it on **AWS SageMaker**, where we will create an endpoint that accepts molecular inputs (SMILES strings, for example). This endpoint will allow users to test new molecular structures at scale by receiving predictions for their antibacterial activity. The cloud infrastructure will enable rapid scaling and parallel processing, making the solution suitable for real-world applications.

### Evaluation and Optimization
The model will be evaluated using metrics such as **ROC-AUC**, **Accuracy**, and **Precision-Recall**. After identifying promising molecular structures, they will undergo iterative optimization using graph-based techniques to refine their drug-likeness, synthetic feasibility, and antibacterial potential further.

## Conclusion

This study serves as the foundation for scaling the discovery of antibiotics through machine learning. By leveraging **Graph Neural Networks (GNNs)** and **AWS SageMaker**, we aim to create a scalable and efficient platform for discovering novel antibiotics. In the future, this system will provide an endpoint through which new molecules can be entered and tested for antibacterial efficacy, contributing to the ongoing fight against antibiotic resistance.

### Data Dictionary for Antibiotic Compounds Dataset

| **Field**            | **Description**                                                                                                                                   |
|----------------------|---------------------------------------------------------------------------------------------------------------------------------------------------|
| **cid**              | The compound identifier (CID), a unique number assigned to the compound in the PubChem database.                                                  |
| **cmpdname**         | The name of the compound.                                                                                                                         |
| **cmpdsynonym**      | Synonyms for the compound. This can include alternate names, trade names, or common names.                                                        |
| **mw**               | The molecular weight of the compound (in grams per mole).                                                                                         |
| **mf**               | The molecular formula of the compound, representing the number and types of atoms.                                                                |
| **polararea**        | The polar surface area of the compound.                                                                                                           |
| **xlogp**            | A measure of the hydrophobicity (logarithm of the partition coefficient between octanol and water) of the compound.                                |
| **heavycnt**         | The number of heavy atoms in the compound (atoms that are not hydrogen).                                                                          |
| **hbonddonor**       | The number of hydrogen bond donors in the compound, which are atoms that can donate hydrogen atoms to form hydrogen bonds.                        |
| **hbondacc**         | The number of hydrogen bond acceptors in the compound, which are atoms that can accept hydrogen atoms to form hydrogen bonds.                     |
| **rotatablebond**    | The number of rotatable bonds in the compound, which indicates the flexibility of the molecule.                                                   |
| **IUPAC Name**       | The International Union of Pure and Applied Chemistry (IUPAC) name of the compound, following standardized chemical nomenclature.                 |
| **Isomeric SMILES**  | The Simplified Molecular Input Line Entry System (SMILES) string, representing the structure of the compound, including stereochemistry information. |
| **InChIKey**         | A hashed version of the InChI string, providing a unique identifier for the compound.                                                             |
| **InChI**            | The International Chemical Identifier (InChI), a textual representation of the compound’s structure.                                              |
| **annotation**       | Additional annotation about the compound, including therapeutic uses and classifications, such as `Anti-Infective Agent`, `Antiprotozoal Agent`, or `Anti-Bacterial Agents`. |


In [ ]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Check if the file exists
file_path = '/content/drive/MyDrive/Colab Notebooks/PubChem_compound_text_Antibiotics.csv'
if os.path.exists(file_path):
    print("File exists and ready to load!")
else:
    print("File not found. Check the file path.")


In [ ]:
import pandas as pd

# Load the CSV file
df = pd.read_csv(file_path)

# Display the first few rows to verify
df.head()


In [ ]:
# Check for missing values
print("Missing values per column:")
print(df.isnull().sum())

# Get basic statistics about the dataset
print("\nBasic statistics:")
print(df.describe())

# Display data types to ensure consistency
print("\nData types:")
print(df.dtypes)

# Check the first few rows to ensure everything is loaded correctly
df.head()


### Imputing Missing xlogp Values Using Predictive Modeling

#### What:
We have found that 17 values in the xlogp column are missing, and the variability in the existing values is quite high. Simple imputation (e.g., mean or median) could introduce bias, as the distribution of xlogp is broad and not concentrated around a single value. Therefore, we will use predictive modeling to estimate these missing values based on other available features in the dataset.

#### Why:
Predictive imputation uses the relationships between the missing values and other features to provide more accurate estimations. In this case, we can build a model to predict xlogp using the rest of the columns as inputs. This approach is more robust than simply filling missing values with a constant, as it takes into account the actual distribution and relationships in the data.

#### How:
1. We will first prepare the dataset, ensuring that no columns used for prediction have missing values themselves.
2. We'll split the dataset into two parts:
    - One with known xlogp values (to train the model).
    - One with missing xlogp values (to impute).
3. A regression model (e.g., RandomForestRegressor) will be trained on the known values to predict xlogp.
4. We will then use this model to predict the missing values and fill them back into the dataset.


In [ ]:
# Import necessary libraries
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np

# Load your dataset
file_path = '/content/drive/MyDrive/Colab Notebooks/PubChem_compound_text_Antibiotics.csv'
data = pd.read_csv(file_path)

# Drop rows where xlogp is missing for training, and separate the rows with missing xlogp for prediction
data_with_xlogp = data.dropna(subset=['xlogp'])
data_without_xlogp = data[data['xlogp'].isnull()]

# Selecting features to use for predicting xlogp (excluding 'xlogp' itself)
features = ['mw', 'polararea', 'heavycnt', 'hbonddonor', 'hbondacc', 'rotbonds', 'exactmass', 'monoisotopicmass']

# Training data
X_train = data_with_xlogp[features]
y_train = data_with_xlogp['xlogp']

# The rows without xlogp, which need to be predicted
X_pred = data_without_xlogp[features]

# Train a Random Forest Regressor model
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Predict the missing xlogp values
predicted_xlogp = rf.predict(X_pred)

# Impute the missing xlogp values with the predicted values
data.loc[data['xlogp'].isnull(), 'xlogp'] = predicted_xlogp

# Check the imputation
print(data[['xlogp']].isnull().sum())  # This should print 0, meaning all missing values have been filled.


In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
# Extracting the features for rows where xlogp is missing
X_missing = data_without_xlogp.drop(['xlogp'], axis=1)

# Ensure that columns in X_train and X_missing are aligned
X_missing = X_missing[X_train.columns]

# Predict the xlogp values for the missing data
predicted_xlogp_missing = rf.predict(X_missing)

# Now plot the original vs predicted xlogp values and highlight the imputed values
plt.figure(figsize=(8, 6))

# Plot the actual vs predicted xlogp values for training data (in blue)
sns.scatterplot(x=y_train, y=rf.predict(X_train), label="Predicted (Train)", color="blue")

# Plot the red circles for imputed xlogp values
sns.scatterplot(x=data_without_xlogp['xlogp'], y=predicted_xlogp_missing,
                label="Imputed xlogp (Red Circles)", color="red", marker="o", s=100)

# Plot the perfect prediction line
plt.plot([y_train.min(), y_train.max()], [y_train.min(), y_train.max()],
         color='green', linestyle='--')

plt.title('Actual vs Predicted xlogp Values (with Imputed Highlighted)')
plt.xlabel('Actual xlogp')
plt.ylabel('Predicted xlogp')
plt.legend()
plt.show()


In [ ]:
# Extracting the features for rows where xlogp is missing
X_missing = data_without_xlogp.drop(['xlogp'], axis=1)

# Ensure that columns in X_train and X_missing are aligned
X_missing = X_missing[X_train.columns]

# Predict the xlogp values for the missing data
predicted_xlogp_missing = rf.predict(X_missing)

# Now plot the actual vs predicted xlogp values and highlight the imputed values
plt.figure(figsize=(8, 6))

# Plot the actual vs predicted xlogp values for training data (in blue)
sns.scatterplot(x=y_train, y=rf.predict(X_train), label="Predicted (Train)", color="blue")

# Plot the red circles for imputed xlogp values
sns.scatterplot(x=predicted_xlogp_missing, y=predicted_xlogp_missing,
                label="Imputed xlogp (Red Circles)", color="red", marker="o", s=100)

# Plot the perfect prediction line
plt.plot([y_train.min(), y_train.max()], [y_train.min(), y_train.max()],
         color='green', linestyle='--')

plt.title('Actual vs Predicted xlogp Values (with Imputed Highlighted)')
plt.xlabel('Actual xlogp')
plt.ylabel('Predicted xlogp')
plt.legend()
plt.show()


In [ ]:
# Select only numeric columns for correlation matrix
numeric_data_with_xlogp = data_with_xlogp.select_dtypes(include=[float, int])

# Calculate the correlation matrix
correlation_matrix = numeric_data_with_xlogp.corr()

# Create a heatmap of the correlation matrix
plt.figure(figsize=(12, 8))
heatmap = sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f", linewidths=0.5)
heatmap.set_title('Correlation Matrix of Key Variables', fontdict={'fontsize': 12}, pad=12)

# Display the plot
plt.show()


# Principal Component Analysis (PCA) and Correlation Simplification

## Why Highly Correlated Variables Provide Redundant Information

In any dataset, when two or more variables are highly correlated, it means that they provide similar information. Correlation measures the linear relationship between variables, and a correlation coefficient close to +1 or -1 indicates that one variable can be approximately predicted from the other. In simpler terms, if two variables are highly correlated (either positively or negatively), they are essentially measuring the same aspect of the data.

### Positive Correlation (+1)
A correlation of +1 indicates a perfect positive relationship, where as one variable increases, the other also increases proportionally. This means the two variables are redundant because they move together in the same direction.

### Negative Correlation (-1)
A correlation of -1 indicates a perfect negative relationship, where as one variable increases, the other decreases proportionally. In this case, the variables are inversely related, but still provide overlapping information.

### Why Redundant Variables Are a Problem
When variables are highly correlated, they introduce **multicollinearity** into statistical models like regression. Multicollinearity can cause problems because:
- **Unstable Coefficients**: The model may struggle to determine which variable is more important, leading to large standard errors in estimated coefficients.
- **Overfitting**: Including too many highly correlated variables increases the risk of overfitting the model to the training data, which may reduce its ability to generalize to new data.

## Purpose of Principal Component Analysis (PCA)

PCA is a dimensionality reduction technique that helps us simplify a dataset with many features (variables) into fewer dimensions, while preserving the essential information. By transforming correlated variables into a new set of uncorrelated variables called **principal components**, PCA allows us to:
1. **Reduce Redundancy**: By combining correlated variables into fewer components, we remove redundant information.
2. **Improve Model Performance**: Simplifying the dataset can help reduce the risk of overfitting and improve the stability of models.
3. **Visualize Data**: PCA also allows us to visualize high-dimensional data in a lower-dimensional space, helping to understand its structure.

## Why We Are Performing This Analysis

In our dataset, we observed that some variables are highly correlated with each other, indicating redundancy. To simplify the analysis, reduce multicollinearity, and create a more efficient model, we are applying PCA. Specifically, we aim to:
- **Reduce the number of features**: By using PCA, we can combine the information from correlated features into fewer principal components without losing critical information.
- **Identify key patterns**: PCA helps us identify the underlying structure of the data and the relationships between variables.
- **Prepare data for modeling**: After applying PCA, we will use the principal components as features in our predictive models, improving their performance by removing redundant information.

## Steps for Applying PCA

1. **Standardize the Data**: Before performing PCA, we need to standardize the data so that each feature has a mean of 0 and a standard deviation of 1. This is important because PCA is sensitive to the variance in data.
2. **Fit PCA**: We will fit PCA to the standardized data and compute the principal components.
3. **Visualize Variance Explained**: PCA will return a set of principal components. We will examine how much variance is explained by each component, helping us decide how many components to keep.
4. **Transform Data**: Finally, we will use the principal components to transform the dataset, replacing the original variables with the new components.


In [ ]:
# Set a correlation threshold
threshold = 0.8

# Find pairs of variables with high correlation
high_corr_pairs = correlation_matrix.unstack().sort_values(kind="quicksort").drop_duplicates()

# Filter out only pairs that exceed the threshold
high_corr_pairs = high_corr_pairs[(high_corr_pairs > threshold) | (high_corr_pairs < -threshold)]

# Display the high correlation pairs
print(high_corr_pairs)


# Key Observations:
1. Mass-related Variables (mw, exactmass, monoisotopicmass): These variables are perfectly correlated with one another (correlation of 1.0). In any subsequent analysis, we can keep just one of these and discard the others to reduce redundancy.

2. Polar Surface Area (polararea): This variable is highly correlated with several other features, including complexity (0.842), hbondacc (0.911), and mw (0.880). We can consider keeping only one of these highly correlated variables.

3. Hydrogen Bond Acceptors (hbondacc): This feature is strongly correlated with many other variables, including heavy atom count (0.899), molecular weight (0.913), and complexity (0.873). This suggests that hbondacc overlaps with a lot of the other variables, and we can consider dropping some.

4. Complexity: Complexity is highly correlated with molecular weight (0.983) and heavy atom count (0.986). Like the mass variables, complexity provides redundant information in combination with these features.

### Why Perform PCA?

Principal Component Analysis (PCA) is a dimensionality reduction technique. It is especially useful in cases where the dataset contains highly correlated features. Highly correlated variables (positive or negative) provide redundant information. PCA transforms the data into a new coordinate system such that the greatest variance by any projection of the data comes to lie on the first principal component (PC1), the second greatest variance on the second component (PC2), and so on.

This allows us to simplify the dataset while retaining most of its variability. By focusing on a reduced number of principal components, we can capture the essential patterns of the data without having to process all of the original features.

In this case, many of our features are highly correlated, and PCA will help us identify the most important features contributing to the variability in our data.


In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import pandas as pd

# Select the features for PCA (excluding 'cid', 'cmpdname', etc.)
selected_features = [
    'mw', 'polararea', 'complexity', 'heavycnt', 'hbonddonor', 'hbondacc',
    'rotbonds', 'exactmass', 'monoisotopicmass', 'charge',
    'covalentunitcnt', 'totalatomstereocnt', 'definedatomstereocnt',
    'undefinedatomstereocnt', 'totalbondstereocnt', 'definedbondstereocnt',
    'undefinedbondstereocnt', 'pclidcnt', 'gpidcnt', 'gpfamilycnt', 'annothitcnt'
]

# Standardize the data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(data_with_xlogp[selected_features])

# Run PCA
pca = PCA()
pca_result = pca.fit_transform(scaled_data)

# Explained variance
explained_variance = pca.explained_variance_ratio_

# Display explained variance per component
for i, variance in enumerate(explained_variance, 1):
    print(f'PC{i}: {variance * 100:.2f}%')

# Plot cumulative explained variance
plt.figure(figsize=(8, 6))
plt.plot(range(1, len(explained_variance) + 1), explained_variance.cumsum(), marker='o', linestyle='--', color='b')
plt.title('Cumulative Explained Variance by Principal Components')
plt.xlabel('Number of Principal Components')
plt.ylabel('Cumulative Explained Variance (%)')
plt.grid(True)
plt.show()

# Optional: Create a DataFrame to explore the PCA components
pca_df = pd.DataFrame(pca_result, columns=[f'PC{i+1}' for i in range(pca.n_components_)])
pca_df.head()


### How to Interpret PCA Results

When performing PCA, the output provides important insights into the structure of the dataset. Here’s how to interpret the results step by step:

#### 1. **Explained Variance Ratio**
   The **explained variance ratio** tells us how much of the total variance in the data is captured by each principal component (PC). Each PC is an orthogonal (uncorrelated) linear combination of the original variables, and the total variance in the dataset is distributed across these PCs.

   - **PC1** will explain the largest amount of variance in the data.
   - **PC2** will explain the second largest, and so on.

   For example, if PC1 has an explained variance ratio of **40%**, that means 40% of the variance in the original data is captured by this component alone.

   **Interpreting the first few PCs:**
   - The first few principal components (usually PC1, PC2, PC3, etc.) are the most important because they capture most of the variability in the data. Components with very low explained variance (e.g., PC10, PC11) are less important and can be ignored if we want to reduce the dimensionality.
   
   You can use a rule of thumb that **selects the number of components** that explain around **85% to 95%** of the variance to retain most of the information.

#### 2. **Cumulative Explained Variance Plot**
   This plot shows how much variance is cumulatively explained by increasing the number of principal components. Typically, the curve will show that most of the variance is captured by the first few components, and then it levels off.

   - If the curve flattens after a certain number of PCs (e.g., after PC4), it indicates that **adding more components beyond that point doesn't significantly increase the explained variance.**
   - For example, if the plot shows that **90% of the variance** is explained by the first **4 components**, you may choose to keep those 4 components and discard the rest, simplifying the dataset without losing too much information.

#### 3. **Principal Component Scores**
   The transformed dataset after applying PCA gives us the **principal component scores** for each observation. These scores represent the coordinates of each observation in the new space defined by the principal components. They can be used for further analysis or as input to machine learning models.

   - **PC1, PC2, etc.** form new axes where each observation is represented as a point in this new space. These axes capture the most significant patterns in the data.
   - **Higher variance along a component** means that component contains more useful information about how the observations vary.

#### 4. **Loadings (Contribution of Original Features to Each PC)**
   Each principal component is a linear combination of the original features. The coefficients (also called **loadings**) for each original feature in these combinations tell us how much each feature contributes to each principal component.

   - **Positive or negative loadings** show the direction of the relationship with that component. Large positive or negative values indicate a strong influence on the component.
   - Features with high loadings on the first few principal components are the most important in explaining the variance in the dataset.

   **Example Interpretation:**
   - If PC1 has high positive loadings for features like `mw` (molecular weight) and `hbonddonor` (hydrogen bond donors), it means that these features are strongly associated with the first principal component.

#### 5. **Dimensionality Reduction**
   After running PCA, you can choose to keep only the principal components that explain a high percentage of variance (e.g., the first 4 or 5). This reduced set of components can then be used for further analysis or modeling.

   - By reducing the dimensionality, we can simplify the data and reduce computational complexity, while still retaining the majority of the original information.

### Practical Steps

1. **Check how many components explain ~85-95% of the variance.** This will help decide how many PCs to retain.
2. **Inspect the loadings** for the first few principal components to understand which features are contributing the most.
3. **Use the reduced dataset** (with the selected number of principal components) for further analysis or as input to machine learning algorithms.


In [ ]:
# Get the explained variance from the PCA
pca_explained_variance = pca.explained_variance_ratio_

# Plot the explained variance for the top PCA components
plt.figure(figsize=(14, 10))
plt.bar(range(1, len(pca_explained_variance)+1), pca_explained_variance, color='blue', alpha=0.7)
plt.xlabel('Principal Components')
plt.ylabel('Explained Variance Ratio')
plt.title('Explained Variance by Principal Components')
plt.xticks(range(1, len(pca_explained_variance)+1))
plt.grid(True)

# Annotate with variance percentages
for i, v in enumerate(pca_explained_variance):
    plt.text(i + 1, v + 0.01, f"{v*100:.2f}%", ha='center', va='bottom')

plt.show()


In [ ]:
# Select only numerical columns
numerical_columns = data_with_xlogp.select_dtypes(include=[np.number])

# Variance of original features (only for numerical columns)
original_variance = numerical_columns.var()

# Plot comparison of variance in original features and PCA components
plt.figure(figsize=(14, 6))

# Plot variance of original features
plt.subplot(1, 2, 1)
original_variance.plot(kind='bar', color='green', alpha=0.7)
plt.title('Variance of Original Features (Numerical)')
plt.xlabel('Features')
plt.ylabel('Variance')
plt.xticks(rotation=90)
plt.grid(True)

# Plot explained variance of PCA components
plt.subplot(1, 2, 2)
plt.bar(range(1, len(pca_explained_variance)+1), pca_explained_variance, color='blue', alpha=0.7)
plt.title('Explained Variance of PCA Components')
plt.xlabel('Principal Components')
plt.ylabel('Explained Variance Ratio')
plt.xticks(range(1, len(pca_explained_variance)+1))
plt.grid(True)

# Annotate with variance percentages for PCA
for i, v in enumerate(pca_explained_variance):
    plt.text(i + 1, v + 0.01, f"{v*100:.2f}%", ha='center', va='bottom')

plt.tight_layout()
plt.show()


### Comparing PCA Results with Original Numerical Features

To ensure an accurate comparison between PCA components and the original dataset, we are focusing on **numerical** features only. The `cmpdname`, `cmpdsynonym`, and other non-numerical columns have been excluded from this analysis.

- **Original Features**: The variance for each numerical feature is plotted to show how much variability each feature captures within the dataset.
- **PCA Components**: The explained variance ratio for each principal component is plotted to demonstrate how well the PCA components summarize the data.

This comparison helps us understand how much information is retained after reducing the dimensionality of the dataset with PCA.


In [ ]:
# Set the number of components to keep (for example, keeping 10 components)
num_components_to_keep = 10  # Adjust this based on your PCA analysis

# Ensure numerical_columns is defined if not already done
numerical_columns = data_with_xlogp.select_dtypes(include=[np.number])  # Select only numerical columns

# Display the principal components without using ace_tools
pca_components = pd.DataFrame(pca.components_[:num_components_to_keep],
                              columns=numerical_columns.columns[:21],  # Ensure correct number of columns
                              index=[f'PC{i+1}' for i in range(num_components_to_keep)])

# Display the principal components DataFrame
print("Principal Components after PCA:")
pca_components



### Principal Components Analysis (PCA) - Results

After running PCA on the dataset, we retain **8 components** that together explain 95% of the variance in the data. Each of these components is a linear combination of the original features, and the amount each feature contributes to each component is given by the loadings.

- **Variance Explained**: The first few components capture the majority of the variance, allowing us to reduce the dimensionality of the data without losing significant information.
- **Loadings**: These indicate how much each original feature contributes to a principal component. High absolute values indicate stronger contributions from that feature.


In [ ]:
# Select the number of principal components to keep
num_components_to_keep = 8

# Update numerical_columns to reflect only those columns used in PCA
used_columns = numerical_columns.columns[:len(pca.components_[0])]

# Extract the top principal components
pca_components_to_keep = pd.DataFrame(pca.components_[:num_components_to_keep],
                                      columns=used_columns,
                                      index=[f'PC{i+1}' for i in range(num_components_to_keep)])

# Only display the top contributing variables for each principal component
# We can display the absolute highest loadings for each component
top_contributing_features = pca_components_to_keep.apply(lambda x: x.nlargest(5).index, axis=1)

# Print the components that we will keep
print("Top contributing features for each principal component:")
print(top_contributing_features)


## The first two principal components (PC1 and PC2) contribute significantly to explaining the variance, and they have 10 key features combined. Here’s a more refined breakdown:

**PC1 captures variability primarily from features such as:**

* rotbonds
* hbondacc
* cid
* complexity
* polararea

**PC2 captures variability from features such as:**

* definedbondstereocnt
* undefinedbondstereocnt
* polararea
* gpidcnt
* gpfamilycnt

The 10 features in PC1 and PC2 give us a broad understanding of which features are influencing the first two principal components the most, which in turn explain a large portion of the variance in the dataset.

In [ ]:
# Import the necessary libraries
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Define the Random Forest Regressor
rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)

# Apply PCA to the selected numerical columns
numerical_columns_for_pca = numerical_columns[['mw', 'polararea', 'complexity', 'xlogp', 'heavycnt',
                                               'hbonddonor', 'hbondacc', 'rotbonds', 'exactmass',
                                               'monoisotopicmass', 'charge', 'covalentunitcnt',
                                               'isotopeatomcnt', 'totalatomstereocnt', 'definedatomstereocnt',
                                               'undefinedatomstereocnt', 'totalbondstereocnt',
                                               'definedbondstereocnt', 'undefinedbondstereocnt', 'pclidcnt',
                                               'gpidcnt']].values  # Remove the feature names here by using .values

# Apply PCA to the data
pca_components = pca.transform(numerical_columns_for_pca)

# Define the target variable (xlogp values)
target = data_with_xlogp['xlogp'].values

# Check the shape to ensure consistency
print(f"PCA Components shape: {pca_components.shape}")
print(f"Target shape: {target.shape}")

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(pca_components, target, test_size=0.2, random_state=42)

# Fit the Random Forest Regressor to the training data
rf_regressor.fit(X_train, y_train)

# Predict on the test set
y_pred = rf_regressor.predict(X_test)

# Evaluate the model performance
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print out the model performance
print(f"Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}")


# Model Performance Analysis

We applied **Principal Component Analysis (PCA)** to reduce the dimensionality of the dataset, followed by training a **Random Forest Regressor** model to predict the partition coefficient (*xlogp*) of the compounds. Below, we discuss the key results and future steps:

### Results:

- **PCA Components Shape**: (326, 21)  
    - After applying PCA, we retained 21 components from the original dataset.
- **Target Shape**: (326,)  
    - The target variable (xlogp) has 326 samples, matching the size of the PCA components.
- **Mean Squared Error (MSE)**: 7.99  
    - The MSE represents the average squared difference between the actual and predicted *xlogp* values. A higher value indicates more significant deviations between predictions and ground truth.
- **R² Score**: 0.58  
    - The R² score measures how much of the variance in the target variable is explained by the model. A score of **0.58** means that approximately **58%** of the variance in *xlogp* is captured by the model.

### Interpretation:

The **R² Score** of 0.58 indicates that the model performs moderately well but leaves room for improvement. Although **58%** of the variation in *xlogp* is explained by the 21 principal components, the **Mean Squared Error** suggests there is still some degree of error in the predictions.

### Suggestions for Improvement:

1. **Hyperparameter Tuning**:  
    The current model uses default settings for the Random Forest Regressor. Hyperparameter tuning (e.g., adjusting the number of trees, maximum depth, and minimum samples per split) may improve model accuracy.
   
2. **Feature Importance & Selection**:  
    Analyzing the **importance** of the PCA components can help identify the most informative components. Reducing the less important ones might streamline the model without sacrificing accuracy.

3. **Alternative Models**:  
    Testing with other models, such as **Gradient Boosting**, **XGBoost**, or even **neural networks**, may offer better predictive performance.

4. **Expanding Dataset**:  
    Increasing the size of the dataset could help the model generalize better, leading to improved predictions on unseen data.

### Conclusion:

The current model, while functional, leaves room for improvement. By tuning hyperparameters, selecting important features, or exploring alternative models, we can likely improve the model's predictive performance. Additionally, expanding the dataset may further enhance accuracy.


# Why Did We Use More Features Than Those Identified by PCA?

When we performed **Principal Component Analysis (PCA)**, we aimed to reduce the dimensionality of the dataset while retaining as much variance as possible. PCA identified **21 principal components**, each being a combination of the original features. These components captured the majority of the variance in the dataset, with **80% of the variance explained** by the first 10 components.

However, we continued to use **more features** (from the original dataset) in the model for several reasons:

### 1. **PCA Transforms Features, Not Eliminates Them**
PCA doesn’t necessarily eliminate features but **transforms** them into linear combinations of the original variables. While we reduced the dataset to 21 components, these components are derived from the original feature set. Therefore, even though we retained 21 principal components, they still encompass information from **all original variables**.

### 2. **PCA Does Not Always Select 'Physical' Features**
PCA creates new variables (principal components) that are combinations of the original features. These new variables may not directly correspond to individual physical features in the dataset, but they contain combinations of the original data that maximize variance. Thus, **using only the features identified by PCA** would omit important information that is embedded in the original features.

### 3. **Model Accuracy and Feature Engineering**
While PCA is useful for reducing dimensionality, we opted to use the broader feature set to **enhance model accuracy**. By using the original features along with PCA-transformed components, we ensure that no important information is lost. This is particularly important when training machine learning models, where leaving out certain features prematurely may lead to reduced accuracy.

### 4. **Exploring All Features for Future Models**
We used the larger set of features because our goal is to eventually improve model accuracy and identify key patterns in the data. By keeping a larger feature set, we leave room for exploring different machine learning models, hyperparameters, and even combinations of the original features with PCA components. This allows for more flexibility in **feature selection** and testing which set of features yields the best model performance.

### Conclusion:
PCA helps us simplify and reduce data complexity, but it doesn’t always mean that fewer features are required. We kept the broader feature set to ensure that our model captures the complete range of patterns in the data, while still benefiting from the dimensionality reduction that PCA offers.


# Let's tune Hyperparameters to see if we can get a better result from our RandomForest Regressor

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

# Define the hyperparameter grid to search over
param_distributions = {
    'n_estimators': [100, 200, 300, 500, 1000],  # Number of trees
    'max_depth': [None, 10, 20, 30, 40, 50],  # Maximum depth of each tree
    'min_samples_split': [2, 5, 10],  # Minimum number of samples required to split a node
    'min_samples_leaf': [1, 2, 4],  # Minimum number of samples required at a leaf node
    'bootstrap': [True, False]  # Whether bootstrap samples are used when building trees
}

# Initialize the Random Forest Regressor
rf_regressor = RandomForestRegressor(random_state=42)

# Setup the RandomizedSearchCV with 5-fold cross-validation
random_search = RandomizedSearchCV(
    estimator=rf_regressor,
    param_distributions=param_distributions,
    n_iter=50,  # Number of different combinations to try
    cv=5,  # 5-fold cross-validation
    verbose=2,  # Show more information while training
    n_jobs=-1,  # Use all available CPU cores
    random_state=42  # For reproducibility
)

# Fit the RandomizedSearchCV
random_search.fit(X_train, y_train)

# Print the best hyperparameters
print("Best Hyperparameters: ", random_search.best_params_)

# Evaluate the tuned model on the test set
y_pred_tuned = random_search.best_estimator_.predict(X_test)
mse_tuned = mean_squared_error(y_test, y_pred_tuned)
r2_tuned = r2_score(y_test, y_pred_tuned)

# Print the performance of the tuned model
print(f"Tuned Mean Squared Error: {mse_tuned}")
print(f"Tuned R^2 Score: {r2_tuned}")


## Hyperparameter Tuning Methodology

In this step, we optimized the performance of our **Random Forest Regressor** model by tuning its hyperparameters using **RandomizedSearchCV**. Hyperparameter tuning is essential because default model settings may not always provide the best possible performance. RandomizedSearchCV automates the process of exploring different combinations of hyperparameters to identify the best configuration.

### Why Hyperparameter Tuning?

Random forests have several key hyperparameters that influence model performance, including:
- **n_estimators**: The number of decision trees in the forest.
- **max_depth**: The maximum depth of each decision tree.
- **min_samples_split**: The minimum number of samples required to split an internal node.
- **min_samples_leaf**: The minimum number of samples required to be at a leaf node.
- **bootstrap**: Whether to use bootstrap samples when building trees.

Tuning these hyperparameters allows us to control the model's complexity, avoid overfitting, and optimize prediction accuracy.

### Method

We used **RandomizedSearchCV** to perform hyperparameter tuning:
1. **RandomizedSearchCV** randomly selects combinations of hyperparameters from a predefined search space. This is more efficient than an exhaustive search and reduces computational cost.
2. We specified the following search space for the random forest:
   - `n_estimators`: Number of trees (between 100 and 500).
   - `max_depth`: Maximum depth of trees (None or between 10 and 50).
   - `min_samples_split`: Minimum number of samples to split a node (between 2 and 10).
   - `min_samples_leaf`: Minimum number of samples required at a leaf node (between 1 and 4).
   - `bootstrap`: Whether bootstrap sampling is used (True or False).
3. **RandomizedSearchCV** was set to perform 5-fold cross-validation, where the data is split into 5 subsets. The model is trained on 4 of the subsets and validated on the remaining one. This process is repeated 5 times with different subsets, ensuring robustness in the evaluation.
4. The best combination of hyperparameters is selected based on the model's performance on the validation set.

### Results

The best hyperparameters identified by **RandomizedSearchCV** were:
- `n_estimators`: 300
- `min_samples_split`: 2
- `min_samples_leaf`: 1
- `max_depth`: None
- `bootstrap`: True

The performance of the tuned model was:
- **Mean Squared Error (MSE)**: 8.23
- **R² Score**: 0.568

### Conclusion

The hyperparameter tuning did not significantly improve the model compared to the default settings. This may suggest that the original settings were already close to optimal. Despite this, hyperparameter tuning is a crucial step in ensuring that we extract the best possible performance from our models. Future steps could involve experimenting with different models or using more advanced techniques such as **GridSearchCV** or ensemble methods.

This is not an unexpected result. Tuning of hyperparamters doesn't always improve the performance of the moderl. However, it is always a 'first stop' along the way because the `RandomizedSearchCV` module can do this quickly and is convenient to use.

## Feature Importance & Selection:

Analyzing the importance of the PCA components can help identify the most informative components. Reducing the less important ones might streamline the model without sacrificing accuracy.

 So let's go ahead and analyze the feature importance of the PCA components to identify the most informative ones. We'll use the feature_importances_ attribute of the Random Forest Regressor to rank the components based on their importance.

Here’s how we can proceed:

**Step-by-step approach:**
1. Train the model using the best hyperparameters (already done).
2. Extract the feature importances from the model.
3. Visualize the importance of each PCA component using a bar plot.
4. Analyze the results and potentially drop the least important components.

In [ ]:
# Refit the model with the best hyperparameters and 8 PCA components (or the number of components kept after PCA)
rf_regressor = RandomForestRegressor(n_estimators=300, min_samples_split=2, min_samples_leaf=1,
                                     max_depth=None, bootstrap=True, random_state=42)

# Fit the Random Forest Regressor to the training data
rf_regressor.fit(X_train[:, :num_components_to_keep], y_train)  # Use only the number of components we retained

# Extract the feature importances from the trained model
feature_importances = rf_regressor.feature_importances_

# Sort the feature importances in descending order and get their corresponding indices
indices = np.argsort(feature_importances)[::-1]

# Plot the feature importances for the retained PCA components
plt.figure(figsize=(10, 6))
plt.title("Feature Importance of PCA Components")
plt.bar(range(num_components_to_keep), feature_importances[indices], align="center")
plt.xticks(range(num_components_to_keep), [f'PC{i+1}' for i in indices], rotation=45)
plt.ylabel("Importance Score")
plt.show()


### Explanation of Feature Importance: Why PC8 and PC6 Are Critical

In the context of our analysis, **PC8** and **PC6** emerged as the most influential principal components in the Random Forest model. Their high importance scores indicate that they capture the most variance in the data that correlates with the target variable, `xlogp`, which represents the partition coefficient of the compounds.

1. **Why PC8 and PC6 Are Important:**
   - PCA works by transforming the original features into a set of orthogonal components (PCs) that represent the directions of maximum variance in the data. The higher the importance of a component, the more strongly it explains the variance associated with the target variable.
   - **PC8** and **PC6** likely capture critical information related to the structural features and chemical properties of the compounds that most affect their partition coefficient.
   - These components may represent hidden interactions between variables like molecular weight, hydrogen bonding, or molecular complexity that are not obvious from the individual original features.

2. **How We Will Use This Data:**
   - **Model Optimization**: We will focus on the most important components (such as PC8 and PC6) to streamline our model, potentially reducing the number of features while maintaining or improving prediction accuracy.
   - **Feature Selection**: The feature importance rankings help us decide which components to prioritize in future iterations of the model. This allows us to ignore less influential components like PC4 or PC2, simplifying the model without significant loss in performance.
   - **Biological Interpretation**: Understanding why certain PCs are important could guide us in interpreting the chemical or structural properties that most affect the antibiotic properties of the molecules. This insight can inform the design of new antibiotics by focusing on the most critical molecular features captured by PC8 and PC6.

In conclusion, the importance of PC8 and PC6 in the model highlights key latent features in the data that we can leverage for more targeted predictions and to refine our feature selection process in future analyses.


Let's start by delving deeper into PC8 and PC6 to understand which original features (chemical properties) contribute most to their importance. The PCA loadings represent how much each feature contributes to each principal component. We'll generate a visualization or table showing the loadings for these components.

**Steps:**

1. Extract PCA Loadings: We'll extract the loading values for each feature in PC8 and PC6.
2. Visualize the Loadings: We'll create a bar chart to visually represent the loadings of each feature in these components. The higher the absolute value of the loading, the more that feature contributes to the principal component.
3. Interpretation: Once we have the loadings, we can analyze the most significant features in PC8 and PC6, which are driving their importance.

Let’s first extract and display the loadings for PC8 and PC6.

PCA Loadings for PC8 and PC6:
We’ll start by generating a table and visualizing the loadings for these components to understand the contribution of each feature.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Ensure the number of columns match the features used in PCA
numerical_columns_for_pca = numerical_columns[['mw', 'polararea', 'complexity', 'xlogp', 'heavycnt',
                                               'hbonddonor', 'hbondacc', 'rotbonds', 'exactmass',
                                               'monoisotopicmass', 'charge', 'covalentunitcnt',
                                               'isotopeatomcnt', 'totalatomstereocnt', 'definedatomstereocnt',
                                               'undefinedatomstereocnt', 'totalbondstereocnt',
                                               'definedbondstereocnt', 'undefinedbondstereocnt', 'pclidcnt',
                                               'gpidcnt']]

# Extract the PCA components into a DataFrame
pca_components_df = pd.DataFrame(pca.components_, columns=numerical_columns_for_pca.columns)

# Extract loadings for PC8 and PC6
pc8_loadings = pca_components_df.iloc[7]  # PC8 is the 8th component (index 7)
pc6_loadings = pca_components_df.iloc[5]  # PC6 is the 6th component (index 5)

# Create a bar chart for PC8 loadings
plt.figure(figsize=(12, 6))
plt.barh(pc8_loadings.index, pc8_loadings.values)
plt.title('Feature Loadings for PC8')
plt.xlabel('Loading Value')
plt.ylabel('Feature')
plt.show()

# Create a bar chart for PC6 loadings
plt.figure(figsize=(12, 6))
plt.barh(pc6_loadings.index, pc6_loadings.values)
plt.title('Feature Loadings for PC6')
plt.xlabel('Loading Value')
plt.ylabel('Feature')
plt.show()

# Display loadings as tables for further inspection
print("Loadings for PC8:")
print(pc8_loadings.sort_values(ascending=False))

print("\nLoadings for PC6:")
print(pc6_loadings.sort_values(ascending=False))


### Interpretation of Principal Components PC8 and PC6:

From the results and visualization, we observe that **PC8** and **PC6** play crucial roles, with specific features contributing significantly to each.

#### Analysis of PC8:
- **Key Contributors**: `gpidcnt`, `covalentunitcnt`, and `heavycnt` have the highest positive contributions, while `definedbondstereocnt` shows the largest negative contribution.
- **Chemical Insights**:
  - **gpidcnt (Gene Product ID Count)** seems to dominate the variance in PC8. This feature is tied to biological properties and can be important for identifying relationships in molecular design.
  - **Covalent Unit Count** and **Heavy Atom Count** are important for structural stability and complexity in molecules.
  - **Defined Bond Stereocenters** show a negative loading, which may indicate a need to minimize certain stereochemistry constraints in future antibiotic designs to optimize the effectiveness of the compounds.

#### Analysis of PC6:
- **Key Contributors**: `totalbondstereocnt` and `isotopeatomcnt` have the highest positive contributions, while `totalatomstereocnt` and `undefinedatomstereocnt` contribute negatively.
- **Chemical Insights**:
  - **Total Bond Stereocenter Count** plays a strong role in PC6, showing the importance of stereochemistry and bond orientation in the chemical structure.
  - **Isotope Atom Count** also contributes, which might suggest an importance in isotopic labeling or diversity in the design of new antibiotics.
  - **Total Atom Stereocenter Count** has a negative loading, indicating that we might want to reduce the stereocenters to simplify molecule design without losing efficacy.

### Next Steps:
We can use this information to design new antibiotics that focus on optimizing or minimizing these chemical properties (i.e., increase `gpidcnt`, simplify stereochemistry) based on their influence on the principal components. These insights can be fed into the **Graph Neural Network (GNN)** for molecular generation, leveraging the importance of specific chemical properties identified through PCA.

We will proceed with this feature analysis in the GNN model to generate potential new antibiotic structures, streamlining based on the insights gathered from **PC8** and **PC6**.


## Overview: Building and Training a Graph Neural Network (GNN) for Antibiotic Discovery

### Objective
The primary goal of this project is to leverage **Graph Neural Networks (GNNs)** to predict molecular properties related to antibiotics. Specifically, we aim to analyze the molecular structure of compounds and use their graph representation to forecast critical properties, such as the partition coefficient (**xlogp**), which can be related to the drug's behavior in biological systems.

This work is part of the larger effort to discover new antibiotics using machine learning, leveraging the power of GNNs to capture the intricate relationships between atoms and bonds in molecular structures.

### Why Use GNNs?
Unlike traditional machine learning models, **GNNs** are particularly well-suited to handle graph-structured data. In chemistry, molecules can naturally be represented as graphs:
- **Atoms** are the **nodes**.
- **Bonds** between atoms are the **edges**.
This graph representation allows GNNs to learn meaningful patterns from molecular structures, which are critical in predicting properties like solubility, reactivity, and drug efficacy.

### Why are we interested in xlogp?
The **xlogp** (partition coefficient) is a key molecular descriptor that indicates how a compound distributes between aqueous (water) and lipid (fat) environments. In drug discovery, understanding a molecule's xlogp value helps in assessing its bioavailability, permeability, and how it behaves in biological systems. Hence, accurately predicting xlogp using GNNs may accelerate the identification of viable antibiotic candidates.

![xlogp](https://cheminfographic.wordpress.com/wp-content/uploads/2020/05/partition-coefficient-logp.jpg?w=1194)


### Steps Taken

1. **Graph Representation of Molecules**:
   Each molecule is represented as a graph, where:
   - **Nodes** represent atoms.
   - **Edges** represent bonds between atoms.

   For each molecule, we create:
   - An **adjacency matrix** to indicate the bonds.
   - **Node features** representing atomic properties (e.g., molecular weight, polarity, etc.).

2. **Feature Selection and PCA**:
   Earlier in our analysis, we identified the most informative features using **Principal Component Analysis (PCA)**, which reduced the complexity of our feature set while retaining most of the variance in the data. These features are used as node properties in the GNN.

3. **Model Setup**:
   We have chosen a **Graph Convolutional Network (GCN)**, a type of GNN, as the architecture for our model. This model takes the molecular graph as input and predicts the xlogp value:
   - **Graph Convolution Layers**: These layers operate over graph-structured data to propagate information between atoms (nodes).
   - **Pooling**: We apply a **global pooling** layer that aggregates the node features into a single representation for each molecule.
   - **Output Layer**: A final dense layer for regression predicts the xlogp value.

4. **Training the Model**:
   We train the model using known molecular data and xlogp values to allow the GNN to learn how to predict the partition coefficient from molecular structures.

### Why This Approach?
The molecular structure plays a crucial role in a drug's effectiveness. By employing GNNs, we leverage the graph-like nature of molecules to make more accurate predictions based on the relationships between atoms. Unlike traditional approaches that treat molecules as flat vectors, GNNs can take advantage of the full structure of the molecule, enabling us to discover new insights and potentially uncover novel antibiotics.

### Expected Outcome
We expect that the GNN model will accurately predict the xlogp values for the molecules in our dataset. By identifying key properties and molecular patterns associated with successful antibiotics, we aim to accelerate the drug discovery process. In the long run, this approach can help to uncover new antibiotic compounds that could combat resistant bacteria and other pathogens.



## 1. Graph Representation Preparation
We'll represent each molecule as a graph where:

* Nodes represent atoms.
* Edges represent bonds (using an adjacency matrix).
For each molecule, we need two things:

An Adjacency Matrix that represents which atoms are connected (i.e., which bonds exist).
Node Features, which are the properties of atoms (such as molecular weight, xlogp, etc.).

We'll need Spektral to handle the GNN part.


In [ ]:
!pip install spektral


# **Prepare Graph Data**

In order to do this we must first:

## Extract Adjacency Matrix and Node Features:

In order to accurately represent molecular structures as graphs, we need to first extract the **adjacency matrices** and **node features** (atomic properties) from the actual molecular data.

Here's how we can approach this:

### Adjacency Matrix:
The adjacency matrix of a molecule represents the bonds between atoms. In this matrix:
- Rows and columns correspond to atoms in the molecule.
- If there is a bond between atom $i$ and atom $j$, then the matrix at position $(i,j)$ will have a value of 1.
- If there is no bond, the value is 0.

### Node Features:
Each node (atom) can have multiple features like atomic number, degree, charge, or molecular weight. These features can be extracted from the molecular structure and used as input into the GNN.

### Steps to Extract Adjacency Matrix and Node Features:

#### Step 1: Extract Molecular Data
If you're using a file or a dataset that contains chemical structures, you can use a chemistry toolkit like **RDKit** to convert SMILES (Simplified Molecular Input Line Entry System) strings into molecular graphs.

#### Step 2: Generate the Adjacency Matrix
Using **RDKit** or a similar tool, you can generate the adjacency matrix for each molecule based on its bonds.

#### Step 3: Extract Node Features
The atomic properties of each atom can be extracted from the molecular structure, providing node features for the GNN.


In [ ]:
!pip install rdkit-pypi


In [ ]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import rdmolops
import numpy as np

# Function to convert a molecule (SMILES) to a graph with adjacency matrix and node features
def molecule_to_graph(smiles):
    # Convert SMILES string to an RDKit molecule object
    mol = Chem.MolFromSmiles(smiles)

    # If the SMILES string is invalid, raise an error
    if mol is None:
        raise ValueError(f"Invalid SMILES string: {smiles}")

    # Get adjacency matrix from the molecule (bonds between atoms)
    adjacency_matrix = rdmolops.GetAdjacencyMatrix(mol)

    # Extract node features (for now, using atomic number, but can be expanded)
    node_features = []
    for atom in mol.GetAtoms():
        atom_features = [atom.GetAtomicNum()]  # Atomic number is used as a node feature
        node_features.append(atom_features)

    # Convert list of node features to a numpy array
    node_features = np.array(node_features)

    return adjacency_matrix, node_features

# Load the dataset that contains SMILES strings (adjust the path to match your dataset location)
# Assuming the dataset has a column named 'isosmiles' containing the SMILES strings
dataset_path = '/content/drive/MyDrive/Colab Notebooks/PubChem_compound_text_Antibiotics.csv'
df = pd.read_csv(dataset_path)

# Placeholder to store graphs (adjacency matrices and node features)
graph_data = []

# Iterate over the dataset and process each molecule's SMILES string
for idx, row in df.iterrows():
    smiles = row['isosmiles']  # Change 'isosmiles' to the actual column name for SMILES strings
    try:
        adj_matrix, node_features = molecule_to_graph(smiles)
        graph_data.append({
            'smiles': smiles,
            'adjacency_matrix': adj_matrix,
            'node_features': node_features
        })
        print(f"Processed molecule {idx + 1}/{len(df)}")
    except ValueError as e:
        print(e)

# Example of how to access the adjacency matrix and node features for the first molecule
first_graph = graph_data[0]
print("First Molecule SMILES:", first_graph['smiles'])
print("First Molecule Adjacency Matrix:\n", first_graph['adjacency_matrix'])
print("First Molecule Node Features:\n", first_graph['node_features'])


### Preparing the Graph Dataset for GNN Input

Now that we have extracted the molecular data, including adjacency matrices and node features, we need to organize this data into a format that is suitable for training a Graph Neural Network (GNN). We will use the TensorFlow GNN library to create a graph dataset.

#### Key Steps:

1. **Graph Representation:**
   - Each molecule is represented as a graph where:
     - Nodes represent atoms.
     - Edges represent bonds between atoms.
   - The adjacency matrix represents the connectivity of the graph.
   - Node features contain atomic properties such as atomic number, charge, or molecular weight.

2. **TensorFlow GNN GraphTensor:**
   - TensorFlow GNN uses `GraphTensor` as the primary data structure to hold graphs.
   - Each `GraphTensor` consists of:
     - `node_sets`: A set of nodes (atoms) with their features.
     - `edge_sets`: A set of edges (bonds) defined by the adjacency matrix.
   
3. **Batching Graphs:**
   - We will batch multiple molecular graphs together for efficient training in the GNN.

#### Objective:
We will convert each molecule's adjacency matrix and node features into a `GraphTensor` format that can be fed into a GNN. This will enable the model to learn molecular representations and properties.

## **Prepare the dataset for the GNN using TensorFlow GNN:**

In [ ]:
!pip install torch
!pip install torch-geometric



### We Are Switching to PyTorch Geometric for GNN Implementation ("Everything Fails All The Time")

❌ In the process of developing code, one has to be ready to deal with incompatibility issues. Here we encounter just such a scenario. But things are still good to go 👌

Initially, we started using **TensorFlow GNN** for building our Graph Neural Network (GNN) model. However, we encountered a version compatibility issue due to the Keras version that TensorFlow GNN requires. Specifically, **TensorFlow GNN requires Keras version 2**, while our current environment uses **Keras version 3**, which leads to incompatibility.

Instead of modifying the environment by reverting to an older version of Keras, we have decided to switch to **PyTorch Geometric (PyG)**. Here are the reasons for this switch:

#### 1. **PyTorch Geometric Is Purpose-Built for Graph Neural Networks**
   - PyTorch Geometric is designed specifically for graph data, providing optimized operations and utilities for handling graph structures such as adjacency matrices, edge lists, and node features.
   - PyG has a large set of pre-built layers and functions that can efficiently handle various GNN architectures, including **GCN, GAT, and GraphSAGE**.

#### 2. **Version Compatibility**
   - Unlike TensorFlow GNN, **PyTorch Geometric works seamlessly with the latest version of PyTorch**, allowing us to avoid compatibility issues.
   - PyG is regularly updated and works well with modern deep learning environments, making it more flexible for our use case.

#### 3. **Ease of Use with Molecular Graph Data**
   - PyTorch Geometric provides efficient ways to handle molecular data, including **adjacency matrices and node features**, which can be easily converted into PyG’s `Data` format.
   - PyG simplifies the creation, training, and evaluation of GNN models, especially for tasks such as molecular property prediction, which is crucial in our project of designing new antibiotics.

#### 4. **Strong Community Support and Documentation**
   - PyTorch Geometric has strong community support, and its documentation provides detailed guides and examples, making it easier for us to develop and troubleshoot our GNN models.
   - There are also plenty of tutorials and open-source resources available for GNN model development using PyG.

#### 5. **Compatibility with Future Tasks**
   - PyTorch Geometric integrates well with other PyTorch functionalities, making it a good choice for more advanced tasks that we may encounter later, such as **transfer learning**, **self-supervised learning**, or more complex GNN architectures.

### TL;DR:
By switching to PyTorch Geometric, we ensure that we have a flexible, optimized, and compatible framework for building and training our GNN models. This will allow us to proceed with the molecular graph data analysis without dealing with versioning issues or other complications.


## 🛑 🛠 Additional Problem: we must adapt the logic for PyTorch Geometric to make it compatible with our dataset of molecular graphs.

**Here's how we can adjust it:**
1. Convert adjacency matrices and node features to PyTorch Geometric's Data format.
2. Batch the graphs and use PyTorch's data loaders to handle graph datasets efficiently.
3. Implement the GNN model using PyG's pre-built layers like GCNConv, GraphConv, or other relevant layers for graph-based tasks.

In [ ]:
!pip install torch-scatter


In [ ]:
!pip install torch-sparse


In [ ]:
!pip install torch-cluster



In [ ]:
!pip install torch-geometric


## Preparing Adjacency Matrices and Node Features

In [ ]:
# Import necessary libraries
import pandas as pd
from rdkit import Chem

# Assuming your dataframe is named 'df' and the SMILES strings are in the 'isosmiles' column
smiles_list = df['isosmiles'].dropna().tolist()  # Drop any NaN values

# Initialize empty lists to hold adjacency matrices and node features
adjacency_matrices = []
node_features_list = []

# Function to convert SMILES to adjacency matrix and node features
def molecule_to_graph(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None, None  # Handle invalid SMILES

    # Get adjacency matrix
    adj_matrix = Chem.GetAdjacencyMatrix(mol)

    # Extract node features (for simplicity, using atomic numbers here)
    node_features = [[atom.GetAtomicNum()] for atom in mol.GetAtoms()]

    return adj_matrix, node_features

# Process the SMILES strings from the dataframe
for i, smiles in enumerate(smiles_list):
    adj_matrix, node_features = molecule_to_graph(smiles)
    if adj_matrix is not None and node_features is not None:
        adjacency_matrices.append(adj_matrix)
        node_features_list.append(node_features)
    print(f"Processed molecule {i+1}/{len(smiles_list)}")



In [ ]:
import torch
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

# Function to convert molecule data to PyTorch Geometric Data format
def create_graph_data(adj_matrix, node_features):
    # Convert adjacency matrix and node features to PyTorch tensors
    edge_index = torch.tensor(list(zip(*torch.where(torch.tensor(adj_matrix) == 1))), dtype=torch.long).t().contiguous()
    x = torch.tensor(node_features, dtype=torch.float)

    # Create the PyTorch Geometric Data object
    graph_data = Data(x=x, edge_index=edge_index)
    return graph_data

# Create a list of Data objects for the entire dataset
graph_data_list = []
for i in range(len(adjacency_matrices)):
    graph_data = create_graph_data(adjacency_matrices[i], node_features_list[i])
    graph_data_list.append(graph_data)

# Batch the graphs for training
loader = DataLoader(graph_data_list, batch_size=32, shuffle=True)

# Print a sample batched graph to verify the format
for batch in loader:
    print(batch)
    break


The output DataBatch(x=[1554, 1], edge_index=[2, 3358], batch=[1554], ptr=[33]) indicates that the molecular graph data has been successfully batched for GNN training.

Here’s what the dimensions represent:

* x=[1554, 1]: This means the total number of nodes across all graphs in the batch is 1,554, and each node has 1 feature (e.g., an atomic feature).
* edge_index=[2, 3358]: This represents the edges between the nodes, where 3,358 pairs of nodes have connections (bonds), and edge_index provides the source and target nodes for each bond.
* batch=[1554]: This tensor assigns each of the 1,554 nodes to one of the 32 graphs in the batch. It helps the model understand which nodes belong to which graph.
* ptr=[33]: This points to the cumulative sum of the number of nodes in each graph, used for efficient batching.

## GNN Model for Classification

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

# Define a simple GCN model for binary classification
class GCNBinaryClassifier(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(GCNBinaryClassifier, self).__init__()
        # First Graph Convolutional Layer
        self.conv1 = GCNConv(input_dim, hidden_dim)
        # Second Graph Convolutional Layer
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        # Output Layer (for binary classification)
        self.fc = torch.nn.Linear(hidden_dim, 1)  # Output dimension is 1 for binary classification

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        # First Graph Convolution + ReLU activation
        x = self.conv1(x, edge_index)
        x = F.relu(x)

        # Second Graph Convolution + ReLU activation
        x = self.conv2(x, edge_index)
        x = F.relu(x)

        # Global pooling and final linear layer
        x = torch.mean(x, dim=0)  # Global mean pooling (for graph-level predictions)
        x = self.fc(x)

        return torch.sigmoid(x)  # Sigmoid activation for binary classification

# Instantiate the model
input_dim = 1  # Number of node features (e.g., atomic features)
hidden_dim = 64  # Adjust based on model complexity
model = GCNBinaryClassifier(input_dim, hidden_dim)


## Adjusting the Training Loop for Classification:
Since it's a classification task, we will use binary cross-entropy loss instead of MSE.

### Define DataLoader for Training and Validation:


In [ ]:
from torch_geometric.loader import DataLoader

# Assuming we have graph_data_list created
# Split data into training and validation sets
train_size = int(0.8 * len(graph_data_list))
val_size = len(graph_data_list) - train_size

train_data = graph_data_list[:train_size]
val_data = graph_data_list[train_size:]

# Create DataLoader for batching
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)


## Add Target Labels to Data Object

In [ ]:
# Update the function to include target (y) in Data object
def create_graph_data(adj_matrix, node_features, target_label):
    # Convert adjacency matrix and node features to PyTorch tensors
    edge_index = torch.tensor(list(zip(*torch.where(torch.tensor(adj_matrix) == 1))), dtype=torch.long).t().contiguous()
    x = torch.tensor(node_features, dtype=torch.float)

    # Create the PyTorch Geometric Data object
    graph_data = Data(x=x, edge_index=edge_index, y=torch.tensor([target_label], dtype=torch.float))
    return graph_data


## Ensure Target Labels are Passed When Creating Graph Data